In [11]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from precision_recall_cutoff import precision_recall_cutoff
from sklearn.feature_selection import RFE, RFECV

s3= boto3.resource('s3')
bucket_name= 'morgan-gant-data448-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'turnover.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [12]:
#changing to dummy variables
turnover= pd.concat([turnover.drop(columns=['sales', 'salary'], axis=1), pd.get_dummies(turnover[['sales', 'salary']])], axis=1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,...,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [14]:
#engineering features from the decisoon tree model
turnover['interaction_1']=np.where(((turnover['satisfaction_level'] >= .115) &
                                   (turnover['satisfaction_level'] >= .465) &
                                   (turnover['number_project'] > 2.5)), 1, 0)

turnover['interaction_2']=np.where(((turnover['satisfaction_level'] >= .465) &
                                   (turnover['number_project'] <=2.5) &
                                   (turnover['last_evaluation'] <= .575)), 1, 0)
        
turnover['interaction3']=np.where(((turnover['satisfaction_level'] >= .465) &
                                    (turnover['time_spend_company'] <=4.5) &
                                    (turnover['average_montly_hours'] <=290.5)), 1,0)

In [15]:
#Defining input and target variables
x= turnover.drop(columns= 'left', axis=1)
y= turnover['left']

#splitting the data
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= .2, stratify= y)

In [16]:
#running RFE with Random forest model
rf_rfecv = RFECV(estimator =RandomForestClassifier(n_estimators= 500, max_depth=3), min_features_to_select = 2, cv=3).fit(x_train, y_train)

#extracting feature names 
print(x_train.columns[rf_rfecv.support_])

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'interaction_1',
       'interaction3'],
      dtype='object')


In [19]:
#defining input and target variables
x_train1= x_train[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'interaction_1',
       'interaction3']]
x_test1= x_test[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'interaction_1', 'interaction3']]

#building the model
rf_md= RandomForestClassifier().fit(x_train1, y_train)

#predicting on the test
rf_pred= rf_md.predict_proba(x_test1)[:,1]

#changing likelyhoods to labels
rf_label= precision_recall_cutoff(y_test, rf_pred)

print(classification_report(y_test,rf_label))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2286
           1       0.99      0.97      0.98       714

    accuracy                           0.99      3000
   macro avg       0.99      0.98      0.99      3000
weighted avg       0.99      0.99      0.99      3000



In [22]:
rf_param_grid = {'n_estimators': [100, 300, 500],
'min_samples_split': [10, 15],
'min_samples_leaf': [5, 7],
'max_depth' : [3, 5, 7]}

#runnning gridsearchCV
rf_grid_search= GridSearchCV(estimator= RandomForestClassifier(), param_grid= rf_param_grid, cv= 5, scoring= 'f1').fit(x_train1, y_train)
    
#Extracting best hyperparamter combo
rf_grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 500}

In [25]:
#building a random forest with best parameters
rf= RandomForestClassifier(n_estimators= 500, max_depth=7, min_samples_leaf= 5, min_samples_split= 10).fit(x_train1, y_train)

#predicting on the test
rf_pred= rf.predict_proba(x_test1)[:,1]

#changing likelyhoods to labels
rf_label= precision_recall_cutoff(y_test, rf_pred)

print(classification_report(y_test,rf_label))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2286
           1       0.98      0.91      0.95       714

    accuracy                           0.98      3000
   macro avg       0.98      0.95      0.97      3000
weighted avg       0.98      0.98      0.98      3000



In [ ]:
#Based on my split and models parameters, the model that wasn't tuned had a better performance then the tuned one. If I ran iterations, I bet it would be the other way around. 